# Lesson 22 - Example: Warehouse Inventory

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Introduction

In this lesson, we will use a multi-table dataset to explore practical applications of joining, grouping, and aggregation operations. The dataset we will work with is a small dataset with fictional data representing the inventory of different products stored at different warehouses owned by a retail chain.

The first DataFrame we will create is `products`. This is a lookup table with information about each product sold by the retailer. This table contains the product ID number, product name, and unit price for each product.

In [0]:
products = spark.createDataFrame(
    data = [[101, 'Widget', 10.99],
            [102, 'Sprocket', 12.49],
            [103, 'Grommet', 9.99],
            [104, 'Doodad', 15.89]],
    schema = 'pid INTEGER, product STRING, price DOUBLE'
)
  
products.show()

+---+--------+-----+
pid| product|price|
+---+--------+-----+
101| Widget|10.99|
102|Sprocket|12.49|
103| Grommet| 9.99|
104| Doodad|15.89|
+---+--------+-----+

We will now create the `warehouses` DataFrame. This is a lookup table with information about each warehouse owned by the retailer. This table contains the warehouse ID number, city, and region for each warehouse.

In [0]:
warehouses = spark.createDataFrame(
    data = [[101, 'St. Louis', 'A'],
            [102, 'Kansas City', 'A'],
            [103, 'Seattle', 'B'],
            [104, 'San Francisco', 'B'],       
            [105, 'Chicago', 'A']],
    schema = 'wid INTEGER, city STRING, region STRING'
)
    
warehouses.show()

+---+-------------+------+
wid| city|region|
+---+-------------+------+
101| St. Louis| A|
102| Kansas City| A|
103| Seattle| B|
104|San Francisco| B|
105| Chicago| A|
+---+-------------+------+

Finally, we will create the `inventory` DataFrame. This table contains the bulk of the data. Each contains one row for each product/warehouse pair for which the given product is present at the given warehouse. The columns in this table represent the warehouse ID number, the product ID number, and the number of units of the product stored at that warehouse.

In [0]:
inventory = spark.createDataFrame(
    data = [[101, 101, 45], [101, 103, 23], [101, 104, 62],
            [102, 102, 12], [102, 103, 31], [103, 102, 54], 
            [103, 104, 24], [104, 101, 34], [104, 102, 23], 
            [104, 103, 18], [104, 104, 29], [105, 101, 57],
            [105, 103, 61]],
    schema = 'wid INTEGER, pid INTEGER, units INTEGER'
)

inventory.show()

+---+---+-----+
wid|pid|units|
+---+---+-----+
101|101| 45|
101|103| 23|
101|104| 62|
102|102| 12|
102|103| 31|
103|102| 54|
103|104| 24|
104|101| 34|
104|102| 23|
104|103| 18|
104|104| 29|
105|101| 57|
105|103| 61|
+---+---+-----+

## Inventory Summary

Our first task will be to restate the information provided in the `inventory` DataFrame, but with city and product names listed as opposed to warehouse and product IDs. We will also provide the total value of each product at each warehouse.

In [0]:
(
    inventory
    .join(warehouses, 'wid', 'left')
    .join(products, 'pid', 'left')
    .select(
        'city', 'product', 'units', 
        expr('ROUND(units * price, 2) AS value')
    )
    .sort('city', 'product')
    .show()
)

+-------------+--------+-----+------+
 city| product|units| value|
+-------------+--------+-----+------+
 Chicago| Grommet| 61|609.39|
 Chicago| Widget| 57|626.43|
 Kansas City| Grommet| 31|309.69|
 Kansas City|Sprocket| 12|149.88|
San Francisco| Doodad| 29|460.81|
San Francisco| Grommet| 18|179.82|
San Francisco|Sprocket| 23|287.27|
San Francisco| Widget| 34|373.66|
 Seattle| Doodad| 24|381.36|
 Seattle|Sprocket| 54|674.46|
 St. Louis| Doodad| 62|985.18|
 St. Louis| Grommet| 23|229.77|
 St. Louis| Widget| 45|494.55|
+-------------+--------+-----+------+

## Total Inventory By Product

We will now determine the total number of units of each type across all warehouses, as well as the total value for units of each product.

In [0]:
( #aggregation
    inventory
    .groupBy('pid')
    .agg(
        expr('SUM(units) AS total_units')
    )
    .join(products, 'pid', 'left')
  #join the lookup table   
    .select(
        'product', 'total_units',
        expr('ROUND(total_units*price, 2) AS total_value')
    )
  #sort  
    .sort('product')
    .show()
)

+--------+-----------+-----------+
 product|total_units|total_value|
+--------+-----------+-----------+
 Doodad| 115| 1827.35|
 Grommet| 133| 1328.67|
Sprocket| 89| 1111.61|
 Widget| 136| 1494.64|
+--------+-----------+-----------+

In [0]:
# NOT recommend. This solution is not ideal
(
    inventory
    .join(products, 'pid', 'left')
    .withColumn('value', expr('units * price'))
    .groupBy('product')
    .agg(
        expr('SUM(units) AS units'), 
        expr('ROUND(SUM(value), 2) AS value')
    )
    .sort('product')
    .show()
)

+--------+-----+-------+
 product|units| value|
+--------+-----+-------+
 Doodad| 115|1827.35|
 Grommet| 133|1328.67|
Sprocket| 89|1111.61|
 Widget| 136|1494.64|
+--------+-----+-------+

## Total Inventory Value by Warehouse

Our next task will be to determine the total value of all products stored at each individual warehouse.

In [0]:
# This solution is more calculation efficient
(
    inventory
    .join(products, 'pid', 'left')
    .withColumn('value', expr('price*units'))
    .groupBy('wid')
    .agg(
        expr('ROUND(SUM(value),2) AS total_value')
    )
    .join(warehouses, 'wid', 'left')
    .select('city', 'total_value')
    .sort('city')

    .show()
)

+-------------+-----------+
 city|total_value|
+-------------+-----------+
 Chicago| 1235.82|
 Kansas City| 459.57|
San Francisco| 1301.56|
 Seattle| 1055.82|
 St. Louis| 1709.5|
+-------------+-----------+

In [0]:
# this solution is not ideal 
(
    inventory
    .join(products, 'pid', 'left')
    .join(warehouses, 'wid', 'left')
    .withColumn('value', expr('units * price'))
    .groupBy('city')
    .agg(
        expr('ROUND(SUM(value), 2) AS value')
    )
    .sort('city')
    .show()
)

+-------------+-------+
 city| value|
+-------------+-------+
 Chicago|1235.82|
 Kansas City| 459.57|
San Francisco|1301.56|
 Seattle|1055.82|
 St. Louis| 1709.5|
+-------------+-------+

## Total Inventory Value by Region

We will now determine the total value of all products stored at warehouses within each region.

In [0]:
warehouses.show()

+---+-------------+------+
wid| city|region|
+---+-------------+------+
101| St. Louis| A|
102| Kansas City| A|
103| Seattle| B|
104|San Francisco| B|
105| Chicago| A|
+---+-------------+------+

In [0]:
(
    inventory
    .join(warehouses, 'wid', 'left')
    .groupBy('region', 'pid')
    .agg(
        expr('SUM(units) AS units')
    )
    .join(products, 'pid', 'left')
    .withColumn('value', expr('price*units'))
    .groupBy('region')
    .agg(
        expr('ROUND(SUM(value),2) AS value')
    )
    .sort('region')
    .show()
)

+------+-------+
region| value|
+------+-------+
 A|3404.89|
 B|2357.38|
+------+-------+

In [0]:
(
    inventory
    .join(products, 'pid', 'left')
    .join(warehouses, 'wid', 'left')
    .withColumn('value', expr('units * price'))
    .groupBy('region')
    .agg(
        expr('ROUND(SUM(value), 2) AS value')
    )
    .sort('region')
    .show()
)

+------+-------+
region| value|
+------+-------+
 A|3404.89|
 B|2357.38|
+------+-------+

## Total Inventory by Region and Product

In the next example, we will determine number of units and the total value for each region/product pair.

In [0]:
(
    inventory
    .join(warehouses, 'wid', 'left')
    .groupBy('pid', 'region')
    .agg(
        expr('SUM(units) AS units')
    )
    .join(products, 'pid', 'left')
    .select(
        'region', 'product', 'units',
        expr('ROUND(units*price, 2) AS value')
    )
    .sort('region', 'product')
    
    .show()
)

+------+--------+-----+-------+
region| product|units| value|
+------+--------+-----+-------+
 A| Doodad| 62| 985.18|
 A| Grommet| 115|1148.85|
 A|Sprocket| 12| 149.88|
 A| Widget| 102|1120.98|
 B| Doodad| 53| 842.17|
 B| Grommet| 18| 179.82|
 B|Sprocket| 77| 961.73|
 B| Widget| 34| 373.66|
+------+--------+-----+-------+

In [0]:
(
    inventory
    .join(products, 'pid', 'left')
    .join(warehouses, 'wid', 'left')
    .withColumn('value', expr('units * price'))
    .groupBy('region', 'product')
    .agg(
        expr('SUM(units) AS units'), 
        expr('ROUND(SUM(value), 2) AS value')
    )
    .sort('region', 'product')
    .show()
)

+------+--------+-----+-------+
region| product|units| value|
+------+--------+-----+-------+
 A| Doodad| 62| 985.18|
 A| Grommet| 115|1148.85|
 A|Sprocket| 12| 149.88|
 A| Widget| 102|1120.98|
 B| Doodad| 53| 842.17|
 B| Grommet| 18| 179.82|
 B|Sprocket| 77| 961.73|
 B| Widget| 34| 373.66|
+------+--------+-----+-------+

## Products Available in Each City

We will now determine the products that are available within each city.

In [0]:
(
    inventory
    .join(products, 'pid', 'left')
    .join(warehouses, 'wid', 'left')
    .select('city', 'product')
    .sort('city', 'product')
    .show()
)

+-------------+--------+
 city| product|
+-------------+--------+
 Chicago| Grommet|
 Chicago| Widget|
 Kansas City| Grommet|
 Kansas City|Sprocket|
San Francisco| Doodad|
San Francisco| Grommet|
San Francisco|Sprocket|
San Francisco| Widget|
 Seattle| Doodad|
 Seattle|Sprocket|
 St. Louis| Doodad|
 St. Louis| Grommet|
 St. Louis| Widget|
+-------------+--------+

In [0]:
(
    inventory
    .join(products, 'pid', 'left')
    .join(warehouses, 'wid', 'left')
    .groupBy('city')
    .agg(expr('COLLECT_LIST(product) AS products_avail'))
    .sort('city')
    .show(truncate=False)
)

+-------------+-----------------------------------+
city |products_avail |
+-------------+-----------------------------------+
Chicago |[Widget, Grommet] |
Kansas City |[Sprocket, Grommet] |
San Francisco|[Widget, Sprocket, Grommet, Doodad]|
Seattle |[Sprocket, Doodad] |
St. Louis |[Widget, Grommet, Doodad] |
+-------------+-----------------------------------+

## Products Not Sold in Each City

For our final task, we will determine which products are unavailable in each of the cities.

In [0]:
(
    warehouses
    .crossJoin(products)
    .join(inventory, ['wid', 'pid'], 'anti')
    .select('city', 'product')
    .sort('city', 'product')
    .show()
)

+-----------+--------+
 city| product|
+-----------+--------+
 Chicago| Doodad|
 Chicago|Sprocket|
Kansas City| Doodad|
Kansas City| Widget|
 Seattle| Grommet|
 Seattle| Widget|
 St. Louis|Sprocket|
+-----------+--------+

In [0]:
(
    warehouses
    .crossJoin(products)
    .join(inventory, ['wid', 'pid'], 'anti')
    .groupBy('city')
    .agg(expr('COLLECT_LIST(product) AS products_not_avail'))
    .sort('city')
    .show(truncate=False)
)

+-----------+------------------+
city |products_not_avail|
+-----------+------------------+
Chicago |[Sprocket, Doodad]|
Kansas City|[Widget, Doodad] |
Seattle |[Widget, Grommet] |
St. Louis |[Sprocket] |
+-----------+------------------+